### **Advanced SQL**

In [ ]:
# Set-up
%load_ext sql
%sql sqlite://
import pandas as pd

In [ ]:
# Choose files Cities.csv and Countries.csv to upload - must be on local computer
# If running notebook on local computer:
#   No need to run this cell (it will generate an error)
#   Make sure data files are in same workspace as notebook
from google.colab import files
uploaded = files.upload()

Saving Cities.csv to Cities.csv
Saving Countries.csv to Countries.csv


In [ ]:
# Create database tables from CSV files
with open('Cities.csv') as f: Cities = pd.read_csv(f, index_col=0)
%sql drop table if exists Cities;
%sql persist Cities
with open('Countries.csv') as f: Countries = pd.read_csv(f, index_col=0)
%sql drop table if exists Countries;
%sql persist Countries

 * sqlite://
Done.
 * sqlite://
(sqlite3.OperationalError) near "persist": syntax error
[SQL: persist Cities]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
 * sqlite://
Done.
 * sqlite://
(sqlite3.OperationalError) near "persist": syntax error
[SQL: persist Countries]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


#### Look at sample of Cities and Countries tables

In [ ]:
%%sql
select * from Cities limit 5

In [ ]:
%%sql
select * from Countries limit 5

### Duplicates, table variables

*Warm-up: Find all cities in the EU with temperature > 15*

In [ ]:
%%sql
select city
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes' and temperature > 15

*Modify previous query to return EU countries that have a city with temperature > 15, remove duplicates*

*Find number of countries that have a city with latitude > 60 (start with country list then fix)*

In [ ]:
%%sql
select country
from Cities
where latitude > 60

*Modify first query to use table variables*

In [ ]:
%%sql
select city
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes' and temperature > 15

*Find all pairs of cities with the same longitude; return the city pairs and their (shared) longitude - notice what's wrong and fix it*

In [ ]:
%%sql
select C1.city, C2.city, C1.longitude
from Cities C1, Cities C2
where C1.longitude = C2.longitude

*Find all pairs of cities that are near each other, i.e., longitude and latitude are both less than 0.5 apart; return city pairs*

In [ ]:
%%sql
select C1.city, C2.city
from Cities C1, Cities C2
where abs(C1.longitude - C2.longitude) < .5
and abs(C1.latitude - C2.latitude) < .5
and C1.city < C2.city

### <font color = 'green'>**Your Turn**</font>

*Find all pairs of cities with the same temperature. Return the city pairs along with their shared temperature. What do you think about the data after seeing the answer?*

In [ ]:
%%sql
YOUR QUERY HERE

### Subqueries in Where clause

*Find all countries in the Countries table with no city in the Cities table*

In [ ]:
%%sql
select country
from Countries
where not exists (select * from Cities
                  where Cities.country = Countries.country)

*Find countries in the EU that have a city with temperature > 15*

In [ ]:
%%sql
select country
from Countries
where EU = 'yes'
and exists (select * from Cities
            where Cities.country = Countries.country
            and temperature > 15)

*Find number of countries that have a city with latitude > 60 (start with country list)*

In [ ]:
%%sql
select country
from Countries
where exists (select * from Cities
              where Cities.country = Countries.country
              and latitude > 60)

*Find the westernmost city; return the city and longitude*

In [ ]:
%%sql
select city, longitude
from Cities C1
where not exists (select * from Cities C2
                  where C2.longitude < C1.longitude)

*Add easternmost to previous query*

*Westernmost city query using = and min*

In [ ]:
%%sql
select city, longitude
from Cities
where longitude = (select min(longitude) from Cities)

*Find all cities whose temperature is more than 50% higher than the average; return the city, country, and temperature, ordered by descending temperature*

In [ ]:
%%sql
select city, country, temperature
from Cities
where temperature > (select avg(temperature) * 1.5 from Cities)
order by temperature desc

*Number of cities in the EU*

In [ ]:
%%sql
select count()
from Cities
where country in (select country from Countries where EU = 'yes')

*Modify previous query to use "not in"*

*Same query using join instead of subquery*

In [ ]:
%%sql
select count()
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes'

*Number of countries with no coastline and a city with longitude < 20*

In [ ]:
%%sql
select count()
from Countries
where coastline = 'no'
and exists (select * from Cities where country = Countries.country
            and longitude < 20)

*Same query using join instead of subquery (see what's wrong and fix it)*

In [ ]:
%%sql
select count()
from Countries, Cities
where Countries.country = Cities.country
and coastline = 'no' and longitude < 20

*Find countries in Countries table with no city in Cities table using join instead of subquery (subquery version repeated first)*

In [ ]:
%%sql
select country
from Countries
where not exists (select * from Cities
                  where Cities.country = Countries.country)

In [ ]:
%%sql
FILL IN

### <font color = 'green'>**Your Turn**</font>

*Find all cities in a country whose population is < 2; return the city and country. First write the query without a subquery.*

In [ ]:
%%sql
YOUR QUERY HERE

*Now write the same query using a subquery instead of a join.*

In [ ]:
%%sql
YOUR QUERY HERE

*Find all countries with no city having a temperature > 6*

In [ ]:
%%sql
YOUR QUERY HERE

*Now try to write the same query without a subquery; can you?*

In [ ]:
%%sql
YOUR QUERY HERE

### Aggregation with Having clause

*Find all countries with average city temperature > 10; return country and average temperature*

In [ ]:
%%sql
select country, avg(temperature)
from Cities
group by country
having avg(temperature) > 10

*Find all countries with more than 5 cities above latitude 50*

In [ ]:
%%sql
select country
from Cities
where latitude > 50
group by country
having count() > 5

*Same query without Having clause*

In [ ]:
%%sql
select distinct country
from Cities C1
where 5 < (select count() from Cities C2
           where C1.country=C2.country
           and latitude > 50)

*Which combinations of EU versus non-EU and coastline versus no-coastline have a minimum population greater than 0.5?*

In [ ]:
%%sql
select EU, coastline, min(population)
from Countries
group by EU, coastline
having min(population) > 0.5

*Find all countries with average city temperature more than 50% higher than the overall average; return country and average temperature*

In [ ]:
%%sql
select country, avg(temperature)
from Cities
group by country
having avg(temperature) > (select 1.5 * avg(temperature) from Cities)

### <font color = 'green'>**Your Turn**</font>

*Find all countries whose average city longitude is lower than the overall average longitude, and whose average city latitude is higher than the overall average latitude. Return the countries. Note: Yes, you can use "and" in Having clauses!*

In [1]:
%%sql
YOUR QUERY HERE

UsageError: Cell magic `%%sql` not found.


### Subqueries in From and Select clauses

*Find all countries with both cold and warm cities -- at least one city with temperature < 9 and one city with temperature > 14*

In [ ]:
%%sql
select distinct C1.country
from Cities C1, Cities C2
where C1.country = C2.country
and C1.temperature < 9 and C2.temperature > 14

*Modify query to also return count of cold and warm cities (then show without column renaming)*

In [ ]:
# (select count() from Cities where country = C1.country and temperature < 9) as numcold,
# (select count() from Cities where country = C1.country and temperature > 14) as numwarm

*Same query using subquery in From clause instead of Select clause*

In [ ]:
%%sql
select Cold.country, numcold, numwarm
from (select country, count() as numcold from Cities
      where temperature < 9 group by country) Cold,
     (select country, count() as numwarm from Cities
      where temperature > 14 group by country) Warm
where Cold.country = Warm.country

### Data modification

*Increase all city temperatures by 10%*

In [ ]:
%%sql
update Cities
set temperature = 1.1 * temperature

In [ ]:
%%sql
select avg(temperature) from Cities

*Increase temperatures another 10% for cities in countries with coastline*

In [ ]:
%%sql
update Cities
set temperature = 1.1 * temperature
where country in (select country from Countries
                  where coastline = 'yes')

*Delete all cities in Turkey*

In [ ]:
%%sql
delete from Cities
where country = 'Turkey'

*Create a new table NonEU containing list of cities (with country) not in the EU*

In [ ]:
%%sql
drop table if exists NonEU;
create table NonEU(city, country);
insert into NonEU
  select city, country from cities
  where country in (select country from Countries
                    where EU = 'no');
select * from NonEU

*Add your city*

In [ ]:
%%sql
insert into NonEU values ('my-city','my-country');
select * from NonEU

### <font color = 'green'>**Your Turn - Advanced SQL on World Cup Data**</font>

In [ ]:
# Upload world cup data files
# Choose files Players.csv and Teams.csv - must be on local computer
from google.colab import files
uploaded = files.upload()
# Then load tables from CSV files
with open('Players.csv') as f: Players = pd.read_csv(f, index_col=0)
%sql drop table if exists Players;
%sql persist Players
with open('Teams.csv') as f: Teams = pd.read_csv(f, index_col=0)
%sql drop table if exists Teams;
%sql persist Teams

TypeError: ignored

#### Look at sample of Players and Teams tables

In [ ]:
%%sql
select * from Players limit 5

In [ ]:
%%sql
select * from Teams limit 5

*1) Find all pairs of teams who have the same number of goalsFor as
each other and the same number of goalsAgainst as each other.
Return the teams and numbers of goalsFor and goalsAgainst.
Make sure to return each pair only once.*

In [ ]:
%%sql
YOUR QUERY HERE

*2) Find all teams with ranking <30 where no player made more than 150 passes. Return the team and ranking.*

In [ ]:
%%sql
YOUR QUERY HERE

*3) Which team has the highest ratio of goalsFor to goalsAgainst?*

In [ ]:
%%sql
YOUR QUERY HERE

*4) Find all team-position pairs where the average number of passes made by players in that position on that team is greater than 150. Return the team-position pairs.*

In [ ]:
%%sql
YOUR QUERY HERE

*5) Find all teams whose defenders averaged more than 150 passes. Return the team and average number of passes by defenders, in descending order of average passes.*

In [ ]:
%%sql
YOUR QUERY HERE

### <font color = 'green'>**Your Turn Extra - Advanced SQL on Titanic Data**</font>

In [ ]:
# Upload Titanic data file
# Choose file Titanic.csv - must be on local computer
from google.colab import files
uploaded = files.upload()
# Then load table from CSV file
with open('Titanic.csv') as f: Titanic = pd.read_csv(f, index_col=0)
%sql drop table if exists Titanic;
%sql persist Titanic

#### Look at sample of Titanic table

In [ ]:
%%sql
select * from Titanic limit 5

*1) Find pairs of passengers who are likely to be twin children: same last name, same age, same embarkation, and age is under 18. Return each pair once, including their last name, first names, age, and embarkation city.*

In [ ]:
%%sql
YOUR QUERY HERE

*2) Which embarkation cities have more than 40 passengers whose age is missing? Reminder: Blanks in SQL tables are given a special value called 'null', and conditions 'A is null' and 'A is not null' can be used in Where clauses to check whether attribute A has the 'null' value.*

In [ ]:
%%sql
YOUR QUERY HERE

*3) Find all classes where the average fare paid by passengers in that class was more than twice the overall average or less than half the overall average.*

In [ ]:
%%sql
YOUR QUERY HERE

*4) What is the average number of passengers per last name? Hint: Requires using a subquery in the From clause*

In [ ]:
%%sql
YOUR QUERY HERE

*5) List each class and its survival rate, i.e., the fraction of passengers in that class who survived. Hints: Use subqueries in the From clause to compute the number of survivers per class and total passengers per class, and force floating point division by multiplying one operand by 1.0*

In [ ]:
%%sql
YOUR QUERY HERE

*6) Modify your previous query to return the survival rate by gender, i.e., of females and of males.*

In [ ]:
%%sql
YOUR QUERY HERE

*7) Now return the survival rate of children versus adults, i.e., of passengers under age 18 versus those 18 or over (ignoring passengers whose age is missing).*

In [ ]:
%%sql
YOUR QUERY HERE

### <font color = 'green'>**Your Turn Extra - SQL Data Modification on Titanic Data**</font>

In [ ]:
# Reload table from CSV file
# NOTE: You may want to reload frequently to reset the data as you
# experiment with modifications
with open('Titanic.csv') as f: Titanic = pd.read_csv(f, index_col=0)
%sql drop table if exists Titanic;
%sql persist Titanic

*1) Subtract 5 from the fare paid by any passenger under the age of 10. Then compute the new average fare. NOTE: You can put multiple SQL statements in one cell separated by a semicolon.*

In [ ]:
%%sql
YOUR CODE HERE

*2) Create a new table called Survivors, containing the last and first names of all passengers who survived. Then count the number of tuples in the new table.*

In [ ]:
%%sql
YOUR CODE HERE

*3) In the Titanic table delete all but the highest-paying passengers.*

In [ ]:
%%sql
YOUR CODE HERE

*4) In what's left of the table after (3), insert a new tuple for yourself. You can decide your class, fare, where you embarked, and whether you survived. Then show the whole table.*

In [ ]:
%%sql
YOUR CODE HERE